In [8]:
import pandas as pd
import glob
import os
from scipy.interpolate import Akima1DInterpolator
from matplotlib import pyplot as plt 
import numpy as np


path = r"./processed_data/charge/"

def get_DTEV(volatge,v_t,vs,ve,cv_p):
    t = np.where((vs<volatge[0:cv_p]) & (volatge[0:cv_p]<ve))[0]
    return v_t[t.max()]-v_t[t.min()]

def get_CCCT(i_s):
    cc_range = 0.97
    times = 20
    slope = 1.0041
    ccc_p = np.array([])

    while ccc_p.size==0:
        times+=1
        cc = np.where(i_s-i_s.max()*cc_range>0)[0]
        condition = np.where(np.diff(i_s)[cc]-slope<0)[0]
        c= cc.min()+condition
        ccc_p = np.append(ccc_p,c).astype(np.int32) 
        for p in ccc_p:
            if(np.average(i_s[np.max([1,p-100]):p])/np.average(i_s[p:np.min([i_s.size,p+100])])>slope):
                ccc_p = p
                break
            elif p==ccc_p[-1]:
                ccc_p = ccc_p.min()

        slope-=0.0001
        cc_range-=0.001
        if times>100:
            ccc_p=0
            return 1
    return ccc_p

def get_CVCT(v):
    cv_range = 0.997
    times = 20
    slope = 1.00007
    ccv_p = np.array([])
    v_s = v[0:int(v.size*0.8)]

    while ccv_p.size==0:
        times+=1
        cv = np.where(v_s-v_s.max()*cv_range>0)[0]
        # print(cv)
        condition = np.where(np.diff(v_s)[cv-1]<0)[0]
        c= cv.min()+condition
        ccv_p = np.append(ccv_p,c-2).astype(np.int32) 
        for p in ccv_p:
            if(np.average(v_s[p:np.min([v_s.size,p+50])]/np.average(v_s[np.max([1,p-50]):p]))>slope):
                ccv_p = p
                break
            elif p==ccv_p[-1]:
                ccv_p = ccv_p.min()
        slope-=0.00001
        cv_range-=0.0001
        if times>100:
            ccv_p=0
            return 1
    return ccv_p

def get_CVDT(d_current):
    return 0

def get_IC_peaks(IC,voltage,vol_p):
    # vol_p = np.where((voltage>=vol_range[0]) & (voltage<=vol_range[1]))[0]
    vol = voltage[vol_p]
    IC = IC[vol_p]
    IC_d = np.diff(IC)
    v_d = np.diff(vol)
    # k1 = np.diff(IC)
    # print(k1)
    k = np.abs(np.diff(IC_d)/np.diff(v_d))/(1+np.power(np.power(IC_d/v_d,2),1.5))[0:-1]

    nor_k = np.linalg.norm(k)
    nor_IC = np.linalg.norm(IC[0:-1])
    K = nor_k*nor_IC

    # k = np.abs(np.diff(IC_d/v_d)/v_d[0:-1])/(1+np.power(np.power(IC_d/v_d,2),1.5))[0:-1]
    # print(k)
    return vol_p[0],k


In [15]:
from scipy.interpolate import Akima1DInterpolator
from matplotlib import pyplot as plt
import plotly.express as px
from EntropyHub import SampEn

import plotly.io as pio
import plotly.graph_objects as go
from scipy.signal import savgol_filter

pio.renderers.default = "browser"
plotly_config = dict({"scrollZoom": True,'modeBarButtonsToAdd':[
                                        'drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})



battery_list = os.listdir(path)
# del battery_list[0:-8]

for battery in battery_list:   
    i=0
    # file_list = glob.glob(path+battery+"/"+f"{battery}_charge_cycle_*")
    file_list = [f"{path}{battery}/{battery}_charge_cycle_{i}.csv" for i in range(1,len(glob.glob(path+battery+"/"+f"{battery}_charge_cycle_*"))+1)]
    cycle = 1
    loop=0
    ccc_arr = np.array([])
    ccv_arr = np.array([])
    v37_419_arr = np.array([])
    v38_419_arr = np.array([])
    v37_41_arr = np.array([])
    v38_41_arr = np.array([])
    tem_arr = np.array([])
    for file in file_list:

        df = pd.read_csv(file)
        df["Voltage_measured"] = df["Voltage_measured"].astype(np.float64).fillna(method='ffill')
        df["Current_measured"] = df["Current_measured"].astype(np.float64).fillna(method='ffill')
        df["Time"] = df["Time"].astype(np.float64)
        tem_arr = np.append(tem_arr,df["Temperature_measured"].mean())
        try:
            if df["Voltage_measured"].size>201:
                vol = df["Voltage_measured"].to_numpy()
                t = df["Time"].to_numpy()
                cur = df["Current_measured"].to_numpy()
                Aki_vol = Akima1DInterpolator(t,vol)
                Aki_cur = Akima1DInterpolator(t,cur)
                A_t = np.array(range(0,int(t.max())*10))/10
                v = Aki_vol(A_t)
                i = Aki_cur(A_t)

                order = 1
                fremlen = 501
                i_s = savgol_filter(i,fremlen,order)
                v_s = savgol_filter(v,fremlen,order)

                v42 = np.where(v_s>=4.2)[0][0]
                # print(v42[0])
                p = np.where((v_s[0:v42+1]>=3.7) & (v_s[0:v42+1]<=4.2))[0]
                vol42 = v_s[p]
                IC = (i_s[p]/3600)[0:-1]/np.diff(vol42)
                IC_s = savgol_filter(IC,fremlen,order)

                ccc_p = get_CCCT(i_s)
                ccc_t = A_t[ccc_p]
                ccv_p = get_CVCT(v_s)
                ccv_t = A_t.max()-A_t[ccv_p]
                v37_419 = get_DTEV(v_s,A_t,3.7,4.19,ccv_p)
                v38_419 = get_DTEV(v_s,A_t,3.8,4.19,ccv_p)
                v37_41 = get_DTEV(v_s,A_t,3.7,4.1,ccv_p)
                v38_41 = get_DTEV(v_s,A_t,3.8,4.1,ccv_p)
                vol_p = np.where((vol42[0:-1]>=4.1) & (vol42[0:-1]<=4.2))[0]
                vol_p1,k = get_IC_peaks(IC_s,vol42[0:-1],vol_p)
                max_k_p = vol_p1+np.argmax(k)
                # print(k)

                ccc_arr = np.append(ccc_arr,ccc_t)
                ccv_arr = np.append(ccv_arr,ccv_t)
                v37_419_arr = np.append(v37_419_arr,v37_419)
                v38_419_arr = np.append(v38_419_arr,v38_419)
                v37_41_arr = np.append(v37_41_arr,v37_41)
                v38_41_arr = np.append(v38_41_arr,v38_41)
                

                #plot current
                if(loop%100==0):
                    fig = px.scatter(df,x="Time",y="Current_measured")
                    fig.update_layout(
                        dragmode='drawopenpath',
                        newshape_line_color='cyan',
                        title_text=f'{battery}_Current_measured_{loop}', 
                    )
                    fig.add_scatter(x=A_t,y=i,name="origin data(Amakima)")
                    fig.add_scatter(x=A_t,y=i_s,name='savgol_filter')
                    fig.add_shape(type="circle",
                        xref="x", yref="y",
                        x0=A_t[ccc_p]-12, y0=i_s[ccc_p]-0.015, x1=A_t[ccc_p]+12, y1=i_s[ccc_p]+0.015,
                        line_color="LightSeaGreen",
                        name="CCCT"
                    )
                    fig.show(config=plotly_config)

                    #plot voltage
                    fig2 = px.scatter(df,x="Time",y="Voltage_measured")
                    fig2.update_layout(
                        dragmode='drawopenpath',
                        newshape_line_color='cyan',
                        title_text=f'{battery}_Voltage_measured_{loop}'
                    )
                    fig2.add_scatter(x=A_t,y=v,name="origin data(Amakima)")
                    fig2.add_scatter(x=A_t,y=v_s,name='savgol_filter')
                    fig2.add_shape(type="circle",
                        xref="x", yref="y",
                        x0=A_t[ccv_p]-32, y0=v_s[ccv_p]-0.01, x1=A_t[ccv_p]+32, y1=v_s[ccv_p]+0.01,
                        line_color="LightSeaGreen",
                        name="CVCT"
                    )
                    fig2.show(config=plotly_config)

                    fig3 = px.line(x=vol42[0:-1],y=IC_s)
                    fig3.add_shape(type="circle",
                        xref="x", yref="y",
                        x0=vol42[max_k_p]-0.01, y0=IC_s[max_k_p]-0.1, x1=vol42[max_k_p]+0.01, y1=IC_s[max_k_p]+0.01,
                        line_color="LightSeaGreen",
                        name="PEAK"
                    )
                    fig3.add_scatter(x=vol42[0:-1][vol_p[0:-2]] ,y=k/1000+25,name='K')
                    fig3.show(config=plotly_config)

                    fig4 = px.line(x=vol42[0:-1][vol_p[0:-2]] ,y=k)
                    fig4.show(config=plotly_config)

            else:
                ccc_arr = np.append(ccc_arr,0)
                ccv_arr = np.append(ccv_arr,0)
                v37_419_arr = np.append(v37_419_arr,0)
                v38_419_arr = np.append(v38_419_arr,0)
                v37_41_arr = np.append(v37_41_arr,0)
                v38_41_arr = np.append(v38_41_arr,0) 
        except Exception as e:
            ccc_arr = np.append(ccc_arr,0)
            ccv_arr = np.append(ccv_arr,0)
            v37_419_arr = np.append(v37_419_arr,0)
            v38_419_arr = np.append(v38_419_arr,0)
            v37_41_arr = np.append(v37_41_arr,0)
            v38_41_arr = np.append(v38_41_arr,0) 
            print(battery)
            print(loop)
            print(str(e))
        
        loop+=1
    
    features = pd.DataFrame(columns=["CCCT","CVCT","V37_419","V38_419","V37_41","V38_41"])
    features["CCCT"] = ccc_arr
    features["CVCT"] = ccv_arr
    features["V37_419"] = v37_419_arr
    features["V38_419"] = v38_419_arr
    features["V37_41"] = v37_41_arr
    features["V38_41"] = v38_41_arr
    features["Temperature"] = tem_arr
    

    features.to_csv(path+battery+"/"+f"{battery}_Features.csv")



KeyboardInterrupt: 

In [ ]:

fig = px.scatter(df,x="Time",y="Current_measured")
fig.update_layout(
    dragmode='drawopenpath',
    newshape_line_color='cyan',
    title_text=f'{battery}_Current_measured_{loop}'
)
fig.add_scatter(x=A_t,y=v)
fig.add_scatter(x=A_t,y=v_s)
# fig.add_shape(type="circle",
#     xref="x", yref="y",
#     x0=A_t[ccc_p]-12, y0=i_s[ccc_p]-0.015, x1=A_t[ccc_p]+12, y1=i_s[ccc_p]+0.015,
#     line_color="LightSeaGreen",
# )
fig.show(config=plotly_config)

In [ ]:
import pandas as pd
import glob
import os
from scipy.interpolate import Akima1DInterpolator
from matplotlib import pyplot as plt 
import numpy as np
from scipy.interpolate import Akima1DInterpolator
from matplotlib import pyplot as plt
import plotly.express as px
from EntropyHub import SampEn
import plotly.io as pio
import plotly.graph_objects as go
from scipy.signal import savgol_filter
path = r"./processed_data/charge/"
battery_list = os.listdir(path)
pio.renderers.default = "browser"
# del battery_list[0:-8]
plotly_config = dict({"scrollZoom": True,'modeBarButtonsToAdd':[
                                        'drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})
fig = go.Figure()
for battery in ["B0005"]:   
    # file_list = glob.glob(path+battery+"/"+f"{battery}_charge_cycle_*")
    file_list = [f"{path}{battery}/{battery}_charge_cycle_{i}.csv" for i in range(1,169)]
    fea = pd.read_csv(path+battery+"/"+f"{battery}_Features.csv")
    tem_arr = np.array([])
    loop=0
    for file in file_list:
        print(file)
        loop+=1
        df = pd.read_csv(file)
        df["Voltage_measured"] = df["Voltage_measured"].astype(np.float64)
        fig.add_trace(go.Scatter(
            y=df["Voltage_measured"],
            name=f'Voltage {file}',
            mode='lines',
            marker_color=f'rgba({loop}, {loop}, 255, .8)'
        ))
        # fig = fig.add_scatter(y=df["Voltage_measured"],fillcolor=)
        fig.update_layout(
            dragmode='drawopenpath',
            newshape_line_color='cyan',
            title_text=f'{battery}_Current_measured_{loop}'
        )
fig.show(config=plotly_config)
    # fea["Temperature"] = tem_arr
    # fea.to_csv(path+battery+"/"+f"{battery}_Features.csv")

In [ ]:
import pandas as pd
import shutil
meta = pd.read_csv("metadata.csv")
chargedata = meta.loc[meta["type"]=="charge"]
dischargedata = meta.loc[meta["type"]=="discharge"]
impedancedata = meta.loc[meta["type"]=="impedance"]
battery_id = chargedata.groupby("battery_id")
# df2 = pd.DataFrame([])
fig = go.Figure()
for bg in ["B0005"]:
    battery = battery_id.get_group(bg)
    cycle = 0
    file = meta[meta["battery_id"]==bg]
    for csvname in battery["filename"]:
        it = meta[meta["filename"]==csvname].index
        if it-1>min(file.index):
            nx = file.loc[it-1,["type"]]
            if (nx["type"] == "charge").bool():
                print(file.loc[it,["filename"]])
                continue
        # df2 = pd.concat(df2,pd.read_csv(f"data/{csvname}")["Voltage_measured"],axis=1)
        if not os.path.isdir(f"processed_data/charge/{bg}"):
            os.makedirs(f"processed_data/charge/{bg}")
        cycle+=1
        df_tmp = pd.read_csv(f"data/{csvname}")
        fig.add_trace(go.Scatter(
            y=df_tmp["Voltage_measured"],
            name=f'Voltage {cycle}',
            mode='lines',
            marker_color=f'rgba({cycle}, {cycle}, 255, .8)'
        ))
        # fig = fig.add_scatter(y=df["Voltage_measured"],fillcolor=)
        fig.update_layout(
            dragmode='drawopenpath',
            newshape_line_color='cyan',
            title_text=f'{battery}_Current_measured_{cycle}'
        )
        
        # cycle+=1
        df_tmp.to_csv("processed_data/charge/{}/{}_charge_cycle_{}.csv".format(bg,bg,cycle))
fig.show(config=plotly_config)
    